<a href="https://colab.research.google.com/github/zw2788/LocalMinimaConstruction/blob/main/LocalMinimaEx1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from typing import Tuple

import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.display import Image

In [3]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/zw2788/LocalMinimaConstruction/main/Ex1.csv")

data.head()

,x_2dvec,y
0,"[2.8,0.4]",1
1,"[3.1,4.3]",1
2,"[0.1,-3.4]",1
3,"[-4.2,-3.3]",1
4,"[-0.5,0.2]",1


In [4]:
features = [
    "x_2dvec",

]
label = "y"

# train test split
X_raw,  Y = data[features].values, data[label].values

#convert string to array
X_raw = np.array([eval(s[0]) for s in X_raw])

# Standardize the input
# Leave blank to match the example in paper

# formatting
Y = Y.reshape((-1, 1))
print(X_raw)
print(Y)
print(X_raw.shape[0])

[[ 2.8  0.4]
 [ 3.1  4.3]
 [ 0.1 -3.4]
 [-4.2 -3.3]
 [-0.5  0.2]
 [-2.7 -0.4]
 [-3.  -4.3]
 [-0.1  3.4]
 [ 4.2  3.2]
 [ 0.4 -0.1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]]
10


Neural Structure:
$M((x_0, x_1)) = \sigma(v_0σ(w_{0,0}x_0 + w_{0,1}x_1 + b_0) + v_1σ(w_{1,0}x_0 + w_{1,1}x_1 + b_1) + c) $

In [5]:
Image(url='https://github.com/zw2788/LocalMinimaConstruction/blob/main/221Sigmoid.png?raw=true')


In [6]:
def sigmoid(x):
    """Calculates sigmoid function."""
    return 1. / (1 + np.exp(-x))

# parameters between input layer and the hidden layer
W_0 = np.random.normal(size=(2, X_raw.shape[1]))
print(f"Shape of W_0 is {W_0.shape}")
print(W_0)
print(W_0.shape[1])
b = np.random.normal(size=(2, 1))
print(f"Shape of b_1 is {b.shape}")

# parameters between hidden layer and output layer
V_0 = np.random.normal(size=(1, W_0.shape[1]))
c = np.random.normal(size=(1, 1))

# calculate the forward propagation
Z_1 = X_raw @ W_0.T
print(f"\nShape of Z_1 is {Z_1.shape}")
print("Samples for Z_1:")
print(Z_1[:5])

A_1 = sigmoid(Z_1 + b.T)
print(f"Shape of A_1 is {A_1.shape}")
print("Samples for A_1:")
print(A_1[:5])

Z_2 = A_1 @ V_0.T
print(f"\nShape of Z_2 is {Z_2.shape}")
print("Samples for Z_2:")
print(Z_2[:5])

A_2 = Y_hat = sigmoid(Z_2 + c.T)
print(f"Shape of A_2 is {A_2.shape}")
print("Samples for A_2:")
print(A_2[:5])



Shape of W_0 is (2, 2)
[[ 1.84534468  0.48952547]
 [-0.23126319  2.06278445]]
2
Shape of b_1 is (2, 1)

Shape of Z_1 is (10, 2)
Samples for Z_1:
[[ 5.36277528  0.17757686]
 [ 7.82552801  8.15305726]
 [-1.47985212 -7.03659346]
 [-9.36588169 -5.83588331]
 [-0.82476724  0.52818848]]
Shape of A_1 is (10, 2)
Samples for A_1:
[[9.86809807e-01 6.77137824e-01]
 [9.98862468e-01 9.99836106e-01]
 [7.39442025e-02 1.54140089e-03]
 [3.00189028e-05 5.10299872e-03]
 [1.33248233e-01 7.48619088e-01]]

Shape of Z_2 is (10, 1)
Samples for Z_2:
[[1.71011447]
 [2.51631748]
 [0.0052655 ]
 [0.01274584]
 [1.87230451]]
Shape of A_2 is (10, 1)
Samples for A_2:
[[0.69027166]
 [0.83307689]
 [0.28834013]
 [0.28987752]
 [0.72383795]]


In [7]:
def forward_prop(
    X_raw: np.array,
    W_0: np.array,
    b: np.array,
    V_0: np.array,
    c: np.array,
) -> Tuple:
    """Performs the forward propagation of the given NN."""
    # Note the NN structure is passed in from outside.
    Z_1 = X_raw @ W_0.T
    A_1 = sigmoid(Z_1 + b.T)

    Z_2 = A_1 @ V_0.T
    A_2 = Y = sigmoid(Z_2 + c.T)

    return A_2, Z_2, A_1, Z_1

Y_hat, _, _, _ = forward_prop(X_raw=X_raw, W_0=W_0, b=b, V_0=V_0, c=c)

In [26]:
def derivatives_by_backprop(
    X_raw: np.array,
    Y: np.array,
    W_0: np.array,
    b: np.array,
    V_0: np.array,
    c: np.array,
) -> Tuple:
    """Calculates the derivatives of the parameters by backforward propagation.

    Here we assume it is a binary classification problem, with sigmoid activation functions.
    """
    # forward propagation
    dW_0, db, dV_0, dc = 0, 0, 0, 0
    Y_hat, Z_2, A_1, Z_1 = forward_prop(X_raw=X_raw, W_0=W_0, b=b, V_0=V_0, c=c)
    n = len(Y_hat)

    loss = -np.mean(np.multiply(Y, np.log(Y_hat)) + np.multiply(1 - Y, np.log(1 - Y_hat)))

    dZ_2 = Y_hat - Y
    dV_0 = dZ_2.T @ A_1 / n
    dc = np.mean(dZ_2.T, axis=1, keepdims=True)

    dZ_1 = np.multiply(dZ_2 @ V_0, np.multiply(A_1, 1 - A_1))
    dW_0 = (dZ_1.T @ X_raw) / n
    db = np.mean(dZ_1.T, axis=1, keepdims=True)

    return dW_0, db, dV_0, dc, loss

dW_0, db, dV_0, dc, loss = derivatives_by_backprop(X_raw=X_raw, Y=Y, W_0=W_0, b=b, V_0=V_0, c=c)

In [27]:
def gradient_descent(
    X_raw: np.array,
    Y: np.array,
    W_0_init: np.array,
    b_init: np.array,
    V_0_init: np.array,
    c_init: np.array,
    learning_rate: float = 0.01,
    epsilon: float = 1e-6,
    verbose: bool = False,
) -> Tuple:
    """Runs gradient descent to fit the NN via backprop."""
    W_0 = W_0_init
    b = b_init
    V_0 = V_0_init
    c = c_init
    losses = [float("inf"), ]
    roc_auc_scores = [0.5, ]

    diff_in_loss = float("inf")
    iteration = 0
    while abs(diff_in_loss) > epsilon:
        iteration += 1
        dW_0, db, dV_0, dc, loss = derivatives_by_backprop(
            X_raw=X_raw, Y=Y, W_0=W_0, b=b, V_0=V_0, c=c
        )

        W_0 -= learning_rate * dW_0
        b  -= learning_rate * db
        V_0 -= learning_rate * dV_0
        c  -= learning_rate * dc

        losses.append(loss)
        diff_in_loss = losses[-1] - losses[-2]

        Y_hat, _, _, _ = forward_prop(X_raw=X_raw, W_0=W_0, b=b, V_0=V_0, c=c)
        roc_auc = roc_auc_score(y_true=Y, y_score=Y_hat)
        roc_auc_scores.append(roc_auc)

        if verbose and iteration % 10 == 0:
            print(loss, roc_auc)
    return W_0, b, V_0, c, losses

In [37]:
# parameters for the first layer
W_0_init = np.array([[1.06,-0.037],[-0.056,1.095]])
b_init = np.array([[-0.051],[-0.0689]])

# parameters for the second layer

V_0_init = np.array([[3.769,-3.72]])
c_init = np.array([[-0.0148]])

#random initial points
#np.random.seed(42)
#W_0_init = np.random.normal(size=(2, X_raw.shape[1]))
#b_init  = np.random.normal(size=(2, 1))
#V_0_init = np.random.normal(size=(1, W_0.shape[1]))
#c_init  = np.random.normal(size=(1, 1))

#initial points from all zero
#W_0_init = np.array([[0.1,0.1],[0.1,0.1]])
#b_init = np.array([[0.1],[0.1]])
#V_0_init = np.array([[0.1,0.1]])
#c_init = np.array([[0.1]])

W_0, b, V_0, c, losses = gradient_descent(
    X_raw=X_raw,
    Y=Y,
    W_0_init=W_0_init,
    b_init =b_init,
    V_0_init=V_0_init,
    c_init =c_init,
    learning_rate=0.1,
    epsilon=1e-6,
    verbose=True,
)
print(W_0.T)
print(b)
print(V_0.T)
print(c)
print(losses)

[[ 1.05997347 -0.05600367]
 [-0.03707572  1.09500426]]
[[-0.05100597]
 [-0.06889346]]
[[ 3.7689839 ]
 [-3.72002224]]
[[-0.014819]]
[inf, 0.5777380445031233, 0.5777380248073104]
